# Discounted Cash Flow (DCF) and EPS Valuation
#### Financial Data and Statements Provided by the financialmodellingprep json API

Valuation is the analytical process of determining the current (or projected) worth of an asset or a company. There are many techniques used for doing a valuation. Here, we evaluate a company using the Discounted Cash Flow and Discount Dividend Model methods.

Discounted cash flow (DCF) is a valuation method used to estimate the value of an investment based on its future cash flows. DCF analysis attempts to figure out the value of an investment today, based on projections of how much money it will generate in the future.

The dividend discount model (DDM) is a quantitative method used for predicting the price of a company's stock based on the theory that its present-day price is worth the sum of all of its future dividend payments when discounted back to their present value.

Financial data is pulled and parsed from **financialmodellingprep.com** using **pd.read_json**. 

**Important**: To use the underlying API you need to get a free API key from https://financialmodelingprep.com/developer/docs/pricing/

All graphs are interactive. You can zoom in or hover over for more information.

Note: This notebook is a simple show case of valuation models using python. In no way do I claim that these are comprehensive. For example, **companies which experience high variability or fluctuations in cashflows or earnings per share may not be accurately predicted with these models**. You must do your own due diligence when it comes to investing and to understand if these models are suitable for the asset you are evaluating.

#### Built With
 * Python 3
 * Plotly

###### Written by: Jake Robinson (https://github.com/jakerobinson19)

In [265]:
#import necessary packages
import pandas as pd
import json
import requests

# iPython display packages for hiding underlying code
from IPython.display import HTML
from IPython.display import display

# Plotly visualization packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [266]:
def format_data(data):
    data.date = data.date.astype('datetime64')
    data = data.drop(['link',
                      'finalLink',
                      'symbol',
                      'acceptedDate'], axis=1).T
    data.columns = data.iloc[0].astype(str).str[:10]
    data = data[1:]
    #del data.columns.name
    
    return(data)

def log_error(msg, json_response):
    print("ERROR: {}".format(msg))
    print("JSON RESPONSE: {}".format(json_response))
    print("Please check your API Key")

def generate_url(ticker, api_key, p_type, statement):
    return('https://financialmodelingprep.com/api/v3/{}/{}?period={}&apikey={}'.format(statement, 
                                                                                       ticker,
                                                                                       p_type,
                                                                                       api_key))


def get_historical_prices(ticker, api_key):
    url = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?&apikey={}'.format(ticker,api_key)
    p = requests.get(url)
    
    prices_json = json.loads(p.text)
    try:
        prices = pd.DataFrame(prices_json["historical"], columns=['date','open','high','low','close'])
        prices = prices.set_index('date')
    
    except:
        log_error("Historical Prices Are Not Available at this time", prices_json)
        prices = None
        
    return(prices)


def get_profile(ticker, api_key):
    url = generate_url(ticker, api_key, '0', 'profile')
    
    profile = pd.read_json(url)
    profile = profile.T
    #del profile.columns.name
    
    return(profile)

def get_quote(ticker, api_key):
    url = generate_url(ticker, api_key, '0', 'quote')
    
    quote = pd.read_json(url)
    quote = quote.T
    #del quote.columns.name
    
    quote.columns = [ticker]
    quote = quote.loc[['name', 'exchange',
                       'price','yearHigh',
                       'yearLow','marketCap',
                       'volume','eps','pe',
                       'sharesOutstanding']]
    quote.index = ['Name','Exchange',
                  'Price','52-wk High',
                  '52 wk Low','Market Cap',
                  'Volume','EPS','P/E',
                  'Shares Outstanding']
    
    return(quote)

def get_balance_sheet(ticker, api_key, p_type, periods): #periods
    url = generate_url(ticker, api_key, p_type, 'balance-sheet-statement')
    
    b_sheet = pd.read_json(url)
    balance_sheet = format_data(b_sheet)
    
    return(balance_sheet.iloc[:, : periods])

def get_income_statement(ticker, api_key, p_type): #periods
    url = generate_url(ticker, api_key, p_type, 'income-statement')
    
    inc_statement = pd.read_json(url)
    income_statement = format_data(inc_statement)
    
    return(income_statement)

def get_cash_flow_statement(ticker, api_key, p_type, periods):
    url = generate_url(ticker, api_key, p_type, 'cash-flow-statement')
    
    cf_statement = pd.read_json(url)
    cashflow_statment = format_data(cf_statement)
    
    return(cashflow_statment.iloc[:, : periods])
    
#res = requests.get(url, timeout=5)
# method used exclusively for hiding the code in the jupyter notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')

display(tag)

### Input Ticker and API Key

In [267]:
ticker = 'AAPL'
api_key = ''

#### DCF INPUTS

In [268]:
# growth rate for cash flows
g_rate = 0.20

# growth years
g_years = 10

# terminal growth rate and period
t_rate = 0.04

# terminal period years
t_years = 10

# discount rate
discount_rate = 0.12

# =========================================

## Stock Information

In [269]:
quote = get_quote(ticker, api_key) 
quote

,AAPL
Name,Apple Inc.
Exchange,NASDAQ
Price,338.312
52-wk High,354.77
52 wk Low,190.3
Market Cap,1466353582080
Volume,43064299
EPS,12.728
P/E,26.5801
Shares Outstanding,4334329699


## Stock Chart

In [270]:
p = get_historical_prices(ticker, api_key)

fig = go.Figure(data=[go.Candlestick(x=p.index,
                open=p['open'],
                high=p['high'],
                low=p['low'],
                close=p['close'])])

fig.update_layout(
    title="{} Stock Chart".format(ticker),
    xaxis_title="Year",
    yaxis_title="Price")

display(tag)
fig.show()

## Balance Sheet

In [271]:
b = get_balance_sheet(ticker, api_key, 'annual', 5)
b

date,2019-09-28,2018-09-29,2017-09-30,2016-09-24,2015-09-26
fillingDate,2019-10-31,2018-11-05,2017-11-03,2016-10-26,2015-10-28
period,FY,FY,FY,FY,FY
cashAndCashEquivalents,48844000000,25913000000,20289000000,20484000000,21120000000
shortTermInvestments,51713000000,40388000000,53892000000,46671000000,20481000000
cashAndShortTermInvestments,100557000000,66301000000,74181000000,67155000000,41601000000
netReceivables,22926000000,23186000000,17874000000,15754000000,16849000000
inventory,4106000000,3956000000,4855000000,2132000000,2349000000
otherCurrentAssets,35230000000,37896000000,31735000000,21828000000,28579000000
totalCurrentAssets,162819000000,131339000000,128645000000,106869000000,89378000000
propertyPlantEquipmentNet,37378000000,41304000000,33783000000,27010000000,22471000000


## Income Statement

In [272]:
income_statement = get_income_statement(ticker, api_key, 'annual')
income_statement

date,2019-09-28,2018-09-29,2017-09-30,2016-09-24,2015-09-26,2014-09-27,2013-09-28,2012-09-29,2011-09-24,2010-09-25,2009-09-26,2008-09-27,2007-09-29,2006-09-30,2005-09-24,2004-09-25,2003-09-27,2002-09-28,2001-09-29
fillingDate,2019-10-31,2018-11-05,2017-11-03,2016-10-26,2015-10-28,2014-10-27,2013-10-30,2012-10-31,2011-10-26,2010-10-27,2009-10-27,2008-11-05,2007-11-15,2006-12-29,2005-12-01,2004-12-03,2003-12-19,2002-12-19,2001-12-21
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenue,260174000000,265595000000,229234000000,215639000000,233715000000,182795000000,170910000000,156508000000,108249000000,65225000000,36537000000,32479000000,24006000000,13931000000,13931000000,8279000000,6207000000,5742000000,5363000000
costOfRevenue,161782000000,163756000000,141048000000,131376000000,140089000000,112258000000,106606000000,87846000000,64431000000,39541000000,23397000000,21334000000,15852000000,9888000000,9888000000,6020000000,4499000000,4139000000,4128000000
grossProfit,98392000000,101839000000,88186000000,84263000000,93626000000,70537000000,64304000000,68662000000,43818000000,25684000000,13140000000,11145000000,8154000000,4043000000,4043000000,2259000000,1708000000,1603000000,1235000000
grossProfitRatio,0.378178,0.383437,0.384699,0.39076,0.400599,0.38588,0.376245,0.438712,0.404789,0.393775,0.359635,0.343145,0.339665,0.290216,0.290216,0.272859,0.275173,0.279171,0.230282
researchAndDevelopmentExpenses,16217000000,14236000000,11581000000,10045000000,8067000000,6041000000,4475000000,3381000000,2429000000,1782000000,1333000000,1109000000,782000000,534000000,534000000,489000000,471000000,446000000,430000000
generalAndAdministrativeExpenses,18245000000,16705000000,15261000000,14194000000,14329000000,11993000000,10830000000,10040000000,7599000000,5517000000,4149000000,3761000000,2963000000,1859000000,1859000000,1421000000,1212000000,1111000000,1138000000
sellingAndMarketingExpenses,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
otherExpenses,1807000000,0,2745000000,1348000000,1285000000,980000000,1156000000,522000000,415000000,155000000,326000000,620000000,599000000,165000000,165000000,23000000,26000000,29000000,11000000


## Cash Flow Statement

In [273]:
cf = get_cash_flow_statement(ticker, api_key, 'quarterly', 8)
cf

date,2019-09-28,2018-09-29,2017-09-30,2016-09-24,2015-09-26,2014-09-27,2013-09-28,2012-09-29
fillingDate,2019-10-31,2018-11-05,2017-11-03,2016-10-26,2015-10-28,2014-10-27,2013-10-30,2012-10-31
period,FY,FY,FY,FY,FY,FY,FY,FY
netIncome,55256000000,59531000000,48351000000,45687000000,53394000000,39510000000,37037000000,41733000000
depreciationAndAmortization,12547000000,10903000000,10157000000,10505000000,11257000000,7946000000,6757000000,3277000000
deferredIncomeTax,340000000,32590000000,-5966000000,-4938000000,-1382000000,-2347000000,-1141000000,-4405000000
stockBasedCompensation,6068000000,5340000000,4840000000,4210000000,3586000000,2863000000,2253000000,1740000000
changeInWorkingCapital,42628000000,-13358000000,-32000000,19095000000,3685000000,-24545000000,10517000000,2093000000
accountsReceivables,245000000,-5322000000,-2093000000,1095000000,611000000,-4232000000,-2172000000,-5551000000
inventory,-289000000,828000000,-2723000000,217000000,-238000000,-76000000,-973000000,-15000000
accountsPayables,-1923000000,9175000000,9618000000,1791000000,5400000000,5938000000,2340000000,4467000000


## Plot of Past Cash Flows

In [274]:
# graph annual cash flows
fig = make_subplots(rows=1, cols=1, start_cell="top-left")

fig.add_trace(go.Bar(name="Cash Flows", 
                     x=cf.columns, 
                     y=cf.iloc[-1], 
                     text=cf.iloc[-1], 
                     textposition='outside', 
                     marker_color='#2ca02c',
                     texttemplate = "$%{text:,.0f}"), 
              row=1, col=1)

fig.update_layout(
    title="{} Cash Flows".format(ticker),
    xaxis_title="Year",
    yaxis_title="Dollars")

display(tag)
fig.show()

## Determine the Average Growth Rate of Cash Flows

Note: you can choose to use this in your DCF calculation explicity in DCF Inputs box, otherwise it does not impact the valuation model

In [275]:
# get average cash flow growth over the past x periods

# obtain the years and cash flow values of past years from the cashflow statement
periods = len(cf.iloc[-1])
cash_flows = list(cf.iloc[-1])

growth_rates = []

# loop through the past cash flows and calculate each years growth
for i in range(periods-1):
    div_g_rate = (cash_flows[-(i+2)]-cash_flows[-(i+1)])/cash_flows[-(i+2)]
    growth_rates.append(div_g_rate)

# sum the growths and divide by the number of years to get the average
avg_g_rate = sum(growth_rates)/periods
print(" {}%".format(round(avg_g_rate*100,2)))

 2.71%


In [276]:
# this is the main time value calculation based on the dcf inputs provided
x = (1+g_rate)/(1+discount_rate)
y = (1+t_rate)/(1+discount_rate)

try:
    z = x * (1-x**g_years)/(1-x) + (x**g_years)*y*(1-y**t_years)/(1-y)
except:
    growth_contrib = 0
    term_contrib = 0
    
    for i in range(g_years):
        growth_contrib += x**(i+1)
        
    for i in range(t_years):
        term_contrib += y**(i+1)
    
    term_contrib = (x**g_years)*term_contrib
        
    z = growth_contrib + term_contrib

z

28.468298270661705

### DCF Valuation

In [277]:
# calculate the free cash flow per share from the cash flow statement
fcf_per_share = cash_flows[0]/quote.loc['Shares Outstanding'][0]
fcf_per_share

13.58826025938642

In [278]:
if fcf_per_share < 0:
    fcf_dcf = 0
else:
    fcf_dcf = fcf_per_share*z
    
print("${}".format(round(fcf_dcf,2)))

$386.83


### EPS Valuation

In [279]:
# get the eps from the stock quote
eps = quote.loc['EPS'][0]
eps

12.728

In [280]:
if eps < 0 :
    eps_dcf = 0
else:
    eps_dcf = eps*z
    
print("${}".format(round(eps_dcf,2)))

$362.34


### Margin Of Safety

In [281]:
if fcf_dcf == 0:
    mos_fcf = -1
else:
    mos_fcf = (fcf_dcf - quote.loc['Price'][0])/fcf_dcf
    
if eps_dcf == 0:
    mos_eps = -1
else:
    mos_eps = (eps_dcf - quote.loc['Price'][0])/eps_dcf

print("DCF: {}%".format(round(mos_fcf*100,2)))
print("EPS Valuation: {}%".format(round(mos_eps*100,2)))

DCF: 12.54%
EPS Valuation: 6.63%


# DDM
### Discount Dividend Model

Note: The classic dividend discount model works best when valuing a mature company that pays a hefty portion of its earnings as dividends, such as a utility company

The DDM has various limitations including its inability to effectively work with high growth stocks. If the company's dividend growth rate exceeds the expected return rate, you cannot calculate a value because you get a negative denominator in the formula. Stocks don't have a negative value. Consider a company with a dividend growing at 20% while the expected return rate is only 5%: in the denominator (r-g), you would have -15% (5% - 20%).

Moreover, if you hope to value a growth stock with the dividend discount model, your valuation will be based on nothing more than guesses about the company's future profits and dividend policy decisions. Most growth stocks don't pay out dividends. Rather, they re-invest earnings into the company with the hope of providing shareholders with returns by means of a higher share price.

In [282]:
# get historical dividend paid out per share
url = 'https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{}?apikey={}'.format(ticker, api_key)

d = requests.get(url)

divs_json = json.loads(d.text)

try:
    dividends = pd.DataFrame(divs_json["historical"], columns=['date','adjDividend']).set_index('date')
    dividends.index = pd.to_datetime(dividends.index, format='%Y/%m/%d')
except:
    log_error("Historical Dividends Are Not Available at this time Or the company has no dividends", divs_json)
    dividends = None

In [283]:
# sometimes stocks dont have dividends (growth stocks), so we cannot use DDM
dividends

,adjDividend
date,
2020-05-08,0.82000
2020-02-07,0.77000
2019-11-07,0.77000
2019-08-09,0.77000
2019-05-10,0.77000
...,...
1988-05-16,0.00286
1988-02-12,0.00286
1987-11-17,0.00286


In [284]:
avg_grate = 0

if dividends is not None:
    
# obtain dividend growth rate
    div_grates = []

    years = sorted(list(set(dividends.index.year)))
    last_yr = years[-1]

    for year in years[:-1]: 
        div_yr1 = dividends[dividends.index.year == year].sum()[0]
    
        if year+1 != last_yr:
            div_yr2 = dividends[dividends.index.year == year+1].sum()[0]
        else:
            div_yr2 = dividends[dividends.index.year == year+1].iloc[0][0]*4
            final_div = div_yr2
    
        div_growth = (div_yr2-div_yr1)/div_yr1
    
        if div_growth > .5 or div_growth < -0.4:
            continue
        else:
            div_grates.append(div_growth)
    
    avg_grate = round(sum(div_grates)/len(div_grates),2)

print("Calculated Average Dividend Growth: {}%".format(avg_grate*100))

Calculated Average Dividend Growth: 8.0%


In [285]:
next_yr_div = 0

if dividends is not None:
    next_yr_div = round(final_div*(1+avg_grate),2)

print("Predicted Dividend: ${}".format(next_yr_div))

Predicted Dividend: $3.54


#### DDM Equation
$D_0/(r-g)$

where

$D_0$ = Next Years Dividend

$r$ = Expected Rate of Return (Discount Rate)

$g$ = Dividend Growth Rate

In [286]:
if dividends is not None:
    if discount_rate == avg_grate:
        discount_rate += 0.01
    
    ddm = next_yr_div/(discount_rate-avg_grate)
    print("DDM Value: ${}".format(ddm))

DDM Value: $88.50000000000001
